In [1]:
pwd

'/home/wsuser/work'

In [2]:
#import the libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='RXyOGA6CfX6LhYRVoTwqch23u_BC91Cps_DySxa3gulg',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'naturaldisastersintensityanalysis-donotdelete-pr-dwszjazetrgx0b'
object_key = 'dataset.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
ls

dataset/


In [6]:
#import the preprocess library of image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [8]:
#loading image data
x_train=train_datagen.flow_from_directory(r'/home/wsuser/work/dataset/train_set',target_size=(64,64),batch_size=5,class_mode="categorical",color_mode='rgb')
x_test=test_datagen.flow_from_directory(r'/home/wsuser/work/dataset/test_set',target_size=(64,64),batch_size=5,class_mode="categorical",color_mode='rgb')

Found 671 images belonging to 4 classes.
Found 136 images belonging to 4 classes.


In [9]:
#initialize the model
model=Sequential()

In [10]:
#adding layers
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [11]:
model.add(Flatten())

In [12]:
#hidden layer
model.add(Dense(units=300,kernel_initializer="random_uniform",activation='relu'))
model.add(Dense(units=300,kernel_initializer="random_uniform",activation='relu'))
model.add(Dense(units=300,kernel_initializer="random_uniform",activation='relu'))
model.add(Dense(units=300,kernel_initializer="random_uniform",activation='relu'))
model.add(Dense(units=300,kernel_initializer="random_uniform",activation='relu'))

In [13]:
#output layer
model.add(Dense(units=4,kernel_initializer="random_uniform",activation='softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 32)         0

In [15]:
#compile the model
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [16]:
#train the model
model.fit_generator(x_train,steps_per_epoch=134,epochs=50,validation_data=x_test,validation_steps=27)

/tmp/wsuser/ipykernel_12255/68272945.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=134,epochs=50,validation_data=x_test,validation_steps=27)


Epoch 1/50
134/134 [==============================] - 51s 375ms/step - loss: 1.3209 - accuracy: 0.2943 - val_loss: 1.9608 - val_accuracy: 0.2519
Epoch 2/50
134/134 [==============================] - 49s 365ms/step - loss: 1.2905 - accuracy: 0.3574 - val_loss: 1.3892 - val_accuracy: 0.2815
Epoch 3/50
134/134 [==============================] - 49s 366ms/step - loss: 1.2868 - accuracy: 0.3033 - val_loss: 2.3089 - val_accuracy: 0.2889
Epoch 4/50
134/134 [==============================] - 49s 367ms/step - loss: 1.2581 - accuracy: 0.3754 - val_loss: 1.2795 - val_accuracy: 0.3778
Epoch 5/50
134/134 [==============================] - 51s 382ms/step - loss: 1.2317 - accuracy: 0.3679 - val_loss: 1.3490 - val_accuracy: 0.2963
Epoch 6/50
134/134 [==============================] - 50s 377ms/step - loss: 1.1950 - accuracy: 0.4219 - val_loss: 1.3054 - val_accuracy: 0.3185
Epoch 7/50
134/134 [==============================] - 50s 373ms/step - loss: 1.1713 - accuracy: 0.4054 - val_loss: 1.1931 - val_ac

In [17]:
#save the model
model.save("disaster.h5")

In [18]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 9.2 MB/s eta 0:00:01


In [19]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"Xg3ZPYea1H7pJ-xxhDDqnZQ5WH3hG91QGLqWsOIcg2gP"
}
client=APIClient(wml_credentials)

In [20]:
def guid_space_name(client,natural_disaster):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==natural_disaster)['metadata']['id'])

In [21]:
space_uid=guid_space_name(client,'natural_disaster')
print("Space UID" +space_uid )

Space UID80ea687c-8a1b-4885-9b93-7b34c52334b7


In [22]:
client.set.default_space(space_uid)

'SUCCESS'

In [23]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [24]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [25]:
!tar -zcvf Natural_disaster_classification.tgz disaster.h5

disaster.h5


In [26]:
model_details=client.repository.store_model(model='Natural_disaster_classification.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:'CNN Model Building',
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [27]:
model_id=client.repository.get_model_id(model_details)

In [28]:
model_id

'19ca6e84-ce8b-48a3-a73b-382f5a7746b7'

In [29]:
client.repository.download(model_id,'disaster.tar.gb')

Successfully saved model content to file: 'disaster.tar.gb'


'/home/wsuser/work/disaster.tar.gb'